## **0. Setup environment**

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-5371946b-bc7a-8404-76f2-94eebc42c1b7)


In [5]:
!tree -d ../../images

../../images
├── Collection1
│   └── BAYC
└── Collection2
    └── EAPES

4 directories


In [6]:
import numpy as np
import time
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import TensorDataset

In [7]:
"""
import random
# Set random seed for reproducibility
manualSeed = 42
#manualSeed = random.randint(1, 10000) # use if you want new results
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Random Seed: ", manualSeed)
"""
pass

### **1. DCGAN**

In [3]:
class DCGAN_2D():

    def __init__(
        self,                   
        dataroot1,              # Root directory for dataset 1
        dataroot2,              # Root directory for dataset 2
        workers = 2,            # Number of workers for dataloader
        batch_size = 128,       # Batch size during training
        image_size = 64,        # Spatial size of training images. All images will be resized to this size using a transformer.
        nc = 3,                 # Number of channels in the training images. For color images this is 3
        nz = 100,               # Size of z latent vector (i.e. size of generator input)
        ngf = 64,               # Size of feature maps in generator
        ndf = 64,               # Size of feature maps in discriminator
        num_epochs = 5,         # Number of training epochs
        lr = 0.0002,            # Learning rate for optimizers
        beta1 = 0.5,            # Beta1 hyperparam for Adam optimizers
        ngpu = 1                # Number of GPUs available. Use 0 for CPU mode.
        ):
        # Hyperparameters etc.
        self.device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
        self.LEARNING_RATE = lr  # could also use two lrs, one for gen and one for disc
        self.BATCH_SIZE = batch_size
        self.IMAGE_SIZE = image_size
        self.CHANNELS_IMG = nc
        self.NOISE_DIM = nz
        self.NUM_EPOCHS = num_epochs
        self.FEATURES_DISC = ndf
        self.FEATURES_GEN = ngf
        self.BETA = beta1

        self.transform = transforms.Compose(
            [
                transforms.Resize((self.IMAGE_SIZE, self.IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(self.CHANNELS_IMG)], [0.5 for _ in range(self.CHANNELS_IMG)]
                ),
            ]
        )
        #DATASETS IN DATAROOTS MUST HAVE THE SAME SIZE OF THE FIRST DIMENTION
        dataset1 = datasets.ImageFolder(root=dataroot1, transform=self.transform)
        dataset2 = datasets.ImageFolder(root=dataroot2, transform=self.transform)
        self.dataset = TensorDataset(dataset1, dataset2)
        self.dataloader = DataLoader(self.dataset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=workers)
        self.gen = self.Generator(self.NOISE_DIM, self.CHANNELS_IMG, self.FEATURES_GEN, ngpu).to(self.device)
        self.disc1 = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC, ngpu).to(self.device)
        self.disc2 = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC, ngpu).to(self.device)
        if (self.device.type == 'cuda') and (ngpu > 1):
            self.gen = nn.DataParallel(self.gen, list(range(ngpu)))
            self.disc1 = nn.DataParallel(self.disc1, list(range(ngpu)))
            self.disc2 = nn.DataParallel(self.disc2, list(range(ngpu)))
        self.gen.apply(self.weights_init)
        self.disc1.apply(self.weights_init)
        self.disc2.apply(self.weights_init)

        # Initialize BCELoss function
        self.criterion = nn.BCELoss()
        # Create batch of latent vectors that we will use to visualize the progression of the generator
        self.fixed_noise = torch.randn(64, self.NOISE_DIM, 1, 1, device=self.device)
        # Establish convention for real and fake labels during training
        self.real_label = 1.
        self.fake_label = 0.
        # Setup Adam optimizers for both G and D
        self.opt_gen = optim.Adam(self.gen.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))
        self.opt_disc1 = optim.Adam(self.disc1.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))
        self.opt_disc2 = optim.Adam(self.disc2.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))

    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)

    def train(self):
        writer_real1 = SummaryWriter(f"logs/real1")
        writer_real2 = SummaryWriter(f"logs/real2")
        writer_fake = SummaryWriter(f"logs/fake")

        # Training Loop

        # Lists to keep track of progress
        img_list = []
        G_losses = []
        G1_losses = []
        G2_losses = []
        D1_losses = []
        D2_losses = []
        iters = 0

        # For each epoch
        for epoch in range(self.NUM_EPOCHS):
            # For each batch in the dataloader
            if epoch % 5 == 0:
                time.sleep(1)
                print("Epochs done: {}".format(epoch))
                time.sleep(1)
            for i, (data1, data2) in enumerate(tqdm(self.dataloader)):

                ############################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                ###########################
                ## Train with all-real batch
                self.disc1.zero_grad()
                self.disc2.zero_grad()
                # Format batch
                real1_cpu = data1[0].to(self.device)
                b1_size = real1_cpu.size(0)
                label1 = torch.full((b1_size,), self.real_label, dtype=torch.float, device=self.device)
                real2_cpu = data2[0].to(self.device)
                b2_size = real2_cpu.size(0)
                label2 = torch.full((b2_size,), self.real_label, dtype=torch.float, device=self.device)
                # Forward pass real batch through D
                output1 = self.disc1(real1_cpu).view(-1)
                output2 = self.disc2(real2_cpu).view(-1)
                # Calculate loss on all-real batch
                errD1_real = self.criterion(output1, label1)
                errD2_real = self.criterion(output2, label2)
                # Calculate gradients for D in backward pass
                errD1_real.backward()
                D1_x = output1.mean().item()
                errD2_real.backward()
                D2_x = output2.mean().item()

                ## Train with all-fake batch
                # Generate batch of latent vectors
                b_size = (b1_size + b2_size) // 2
                noise = torch.randn(b_size, self.NOISE_DIM, 1, 1, device=self.device)
                # Generate fake image batch with G
                fake = self.gen(noise)
                label1.fill_(self.fake_label)
                label2.fill_(self.fake_label)
                # Classify all fake batch with D
                output1 = self.disc1(fake.detach()).view(-1)
                output2 = self.disc2(fake.detach()).view(-1)
                # Calculate D's loss on the all-fake batch
                errD1_fake = self.criterion(output1, label1)
                errD2_fake = self.criterion(output2, label2)
                # Calculate the gradients for this batch, accumulated (summed) with previous gradients
                errD1_fake.backward()
                D1_G_z1 = output1.mean().item()
                errD2_fake.backward()
                D2_G_z1 = output2.mean().item()
                # Compute error of D as sum over the fake and the real batches
                errD1 = errD1_real + errD1_fake
                errD2 = errD2_real + errD2_fake
                # Update D
                self.opt_disc1.step()
                self.opt_disc2.step()

                ############################
                # (2) Update G network: maximize log(D(G(z)))
                ###########################
                self.gen.zero_grad()
                label1.fill_(self.real_label)  # fake labels are real for generator cost
                label2.fill_(self.real_label)  # fake labels are real for generator cost
                # Since we just updated D, perform another forward pass of all-fake batch through D
                output1 = self.disc1(fake).view(-1)
                output2 = self.disc2(fake).view(-1)
                # Calculate G's loss based on this output
                errG1 = self.criterion(output1, label1)
                errG2 = self.criterion(output2, label2)
                # Calculate gradients for G
                errG1.backward()
                D1_G_z2 = output1.mean().item()
                errG2.backward()
                D2_G_z2 = output2.mean().item()
                # Update G
                self.opt_gen.step()

                # Save Losses for plotting later
                G_losses.append(errG1.item() + errG2.item())
                G1_losses.append(errG1.item())
                G2_losses.append(errG2.item())
                D1_losses.append(errD1.item())
                D2_losses.append(errD2.item())

                # Check how the generator is doing by saving G's output on fixed_noise
                if (iters % 100 == 0) or ((epoch == self.NUM_EPOCHS-1) and (i == len(self.dataloader)-1)):
                    with torch.no_grad():
                        no_grad_fake = self.gen(self.fixed_noise).detach().cpu()
                    img_list.append(vutils.make_grid(no_grad_fake, padding=2, normalize=True))

                    img_grid_real1 = torchvision.utils.make_grid(data1[0][:32], normalize=True)
                    img_grid_real2 = torchvision.utils.make_grid(data2[0][:32], normalize=True)
                    img_grid_fake = torchvision.utils.make_grid(no_grad_fake[:32], normalize=True)

                    writer_real1.add_image("Real", img_grid_real1, global_step=iters)
                    writer_real2.add_image("Real", img_grid_real2, global_step=iters)
                    writer_fake.add_image("Fake", img_grid_fake, global_step=iters)

                iters += 1
            time.sleep(1)
            print('[%d/%d][%d/%d]\tLoss_D1: %.4f\tLoss_G1: %.4f\tD1(x): %.4f\tD1(G(z)): %.4f / %.4f'
                  % (epoch, self.NUM_EPOCHS, i, len(self.dataloader),
                     errD1.item(), errG1.item(), D1_x, D1_G_z1, D1_G_z2))
            time.sleep(1)
            print('[%d/%d][%d/%d]\tLoss_D2: %.4f\tLoss_G2: %.4f\tD2(x): %.4f\tD2(G(z)): %.4f / %.4f'
                  % (epoch, self.NUM_EPOCHS, i, len(self.dataloader),
                     errD2.item(), errG2.item(), D2_x, D2_G_z1, D2_G_z2))
            time.sleep(1)
        return img_list, G_losses, G1_losses, G2_losses, D1_losses, D2_losses

    class Generator(nn.Module):
        def __init__(self, channels_noise, channels_img, features_g, ngpu):
            super(DCGAN_2D.Generator, self).__init__()
            self.ngpu = ngpu
            self.gen = nn.Sequential(
                # input is Z, going into a convolution
                nn.ConvTranspose2d(channels_noise, features_g * 8, 4, 1, 0, bias=False),
                nn.BatchNorm2d(features_g * 8),
                nn.ReLU(True),
                # state size. (features_g*8) x 4 x 4
                nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g * 4),
                nn.ReLU(True),
                # state size. (features_g*4) x 8 x 8
                nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g * 2),
                nn.ReLU(True),
                # state size. (features_g*2) x 16 x 16
                nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_g),
                nn.ReLU(True),
                # state size. (features_g) x 32 x 32
                nn.ConvTranspose2d(features_g, channels_img, 4, 2, 1, bias=False),
                nn.Tanh()
                # state size. (channels_img) x 64 x 64
            )

        def forward(self, x):
            return self.gen(x)

    class Discriminator(nn.Module):
        def __init__(self, channels_img, features_d, ngpu):
            super(DCGAN_2D.Discriminator, self).__init__()
            self.ngpu = ngpu
            self.disc = nn.Sequential(
                # input is (nc) x 64 x 64
                nn.Conv2d(channels_img, features_d, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf) x 32 x 32
                nn.Conv2d(features_d, features_d * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_d * 2),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*2) x 16 x 16
                nn.Conv2d(features_d * 2, features_d * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_d * 4),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*4) x 8 x 8
                nn.Conv2d(features_d * 4, features_d * 8, 4, 2, 1, bias=False),
                nn.BatchNorm2d(features_d * 8),
                nn.LeakyReLU(0.2, inplace=True),
                # state size. (ndf*8) x 4 x 4
                nn.Conv2d(features_d * 8, 1, 4, 1, 0, bias=False),
                nn.Sigmoid()
            )

        def forward(self, x):
            return self.disc(x)

SyntaxError: invalid syntax (2690582924.py, line 46)

In [ ]:
DCGAN_2D = DCGAN_2D(['../../images/Collection1', '../../images/Collection2'])

In [ ]:
img_list, G_losses, G1_losses, G2_losses, D1_losses, D2_losses = DCGAN_2D.train()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(DCGAN.dataloader))

In [ ]:
# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(DCGAN.device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(2,1,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(DCGAN.device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(2,1,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()